<a href="https://colab.research.google.com/github/kareemrasheed89/tweepy/blob/master/twitter_sentiment_bertModel_Copy2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# installing necessary libraries
!pip install config --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install tweepy --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
!pip install nltk --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install wordcloud --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 435 kB 34.3 MB/s 
  Attempting uninstall: wordcloud
    Found existing installation: wordcloud 1.5.0
    Uninstalling wordcloud-1.5.0:
      Successfully uninstalled wordcloud-1.5.0


In [5]:
!pip install transformers --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 37.1 MB/s 
     |████████████████████████████████| 596 kB 39.1 MB/s 
     |████████████████████████████████| 6.6 MB 43.3 MB/s 
     |████████████████████████████████| 101 kB 13.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [6]:
# importing `config.py` to access its variables and other needed dependencies
import config
from tweepy import Client
import tweepy
import pandas as pd
import re
import sys
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import time
from wordcloud import WordCloud
%matplotlib inline
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

In [7]:
#getting recent tweets data
def getTweets():
    #asking for the search term and the desired number of tweets
    global keyword 
    keyword = input('insert search term : ')
    #what is the research query e.g. 
    query = f'{keyword} -is:retweet lang:en'# place_country:GB' #no retweets, language must be English and Country UK.
    
    #input the number tweets required.
    num_tweets = int(input('how many tweets do you want? '))
    #connecting to the twitter API using clent and the bearer_token credentials from config.py
    #client = Client(config.bearer_token)
    
    client = tweepy.Client(bearer_token='AAAAAAAAAAAAAAAAAAAAAGBqVgEAAAAAUvY9oOpJTR8DUWV4sjAQVYj06BA%3Dy0A2A7CX5E9XCqwaYi8pFdkMqk2GqLnBv8Phd2Y5XTv6hJ5ZSx')
    
    # Replace with time period of your choice
    start_time = '2021-01-01T00:00:00Z'

    # Replace with time period of your choice
    end_time = '2021-03-31T00:00:00Z'

    #using tweepy paginator to get over 100 last tweets from twitter api
    tweets = []
    for tweet in tweepy.Paginator(client.search_all_tweets,
                                    query = query,                             
                                    tweet_fields = ['id','created_at', 'text'],#, 'geo'], 
                                    start_time = start_time, end_time = end_time,
                                    max_results = 100).flatten(limit=num_tweets):
        time.sleep(1)
    
        tweets.append(tweet)
    if len(tweets) <= 20:
        print("The search for " + keyword + " returned an insignificant number of tweets; script will terminate")
        sys.exit("The search for " + keyword + " returned an insignificant number of tweets; script will terminate")
    else:
        return tweets


#function to clean the tweets and load them into a DataFrame
def tweetsETL(tweets):
    
    result = []
    
    
    #regex function to clean the tweet text from haashtags, mentions and links
    def cleanTweets(text):
        clean_text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text.lower()).split())
        #clean_text = [x for x in clean_text if x in unwanted_words]
        return clean_text
    
    #function to unpack the tweets list into a dataframe
    for tweet in tweets:
            print(tweet)
            result.append({'id': tweet.id,
                           'text': tweet.text,
                           'clean_tweet' : cleanTweets(tweet.text),
                           'created_at': tweet.created_at,
                           #'lang' : tweet.lang,
                           #'geo' : tweet.geo['place_id']
                           #'source':tweet.source,
                           #'retweets': tweet.public_metrics['retweet_count'],
                           #'replies': tweet.public_metrics['reply_count'],
                           #'likes': tweet.public_metrics['like_count'],
                           #'quote_count': tweet.public_metrics['quote_count']
                      })
    #result = [x for x in result if x not in unwanted_words]
    df = pd.DataFrame(result)
    return df

In [8]:
#using a transformers model "bert" to perform the sentiment analysis on the clean_tweets column.
def sentimentAnalysis(df):
    tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
    model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
    classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
    res = df['clean_tweet'].apply(lambda x: classifier(x[:512]))
    #print(res)
    return res

## function to add the list resulting from the analysis to the original dataframe as score, sentiment and stars
#the sentiment is either negative, positive or neutral, and the number of stars go from 1 to 5
#1 being the most negative sentiment and 5 being the most positive
def sentimentToDf(df,res):
    
    
    tweets_stars = []
    tweets_scores = []
    tweets_sentiment = []
    #looping over the list of result to unpack it into the original tweets dataframe
    for i in range(res.size):
        tweets_stars.append(int(float(res[i][0]['label'].split()[0])))
        tweets_scores.append(res[i][0]['score'])
        if res[i][0]['label'] == '4 stars' or res[i][0]['label'] == '5 stars':
            tweets_sentiment.append('positive')
        elif res[i][0]['label'] == '1 star' or res[i][0]['label'] == '2 stars':
            tweets_sentiment.append('negative')
        else :
            tweets_sentiment.append('neutral')
    df['scores'] = tweets_scores
    df['sentiment'] = tweets_sentiment  
    df['stars'] = tweets_stars
    
    #----------------------------------4/2/2022--------------------------------------------------------------#
    #list frequent keywords to be filtered from the wordcloud
    unwanted_words = ['vaccine', 'vaccines', 'vaccination', 'covid', 'covid-19', 'covid19', 'people', 'amp']
    
    #remove most frequent keywords and stopwords
    filtered_tweets = []
    stop_words = set(stopwords.words('english'))
    for rmv in df['clean_tweet']:
        remove_keywords = [x for x in rmv.split() if x not in unwanted_words]
        remove_keywords = [x for x in remove_keywords if x not in stop_words]
        #print(remove_keywords)
        filtered_tweets.append(str(remove_keywords).replace("'",""))
    
    #update clean_tweets column in the dataframe
    df['clean_tweet'] = filtered_tweets
    #-------------------------------------------------------------------------------------------------------#
    
    #write to csv file
    df.to_csv('fileok.csv')
    return df

#fucntion to Create the wordclouds using data frpom a column of a dataframe
def creatWordCloud(df,clm_name):
    text = " ".join(line for line in df[clm_name])
    # Create the wordcloud object
    wordcloud = WordCloud(width=980, height=580, margin=0,collocations = False, background_color = 'white').generate(text)
    # Display the generated image:
    plt.figure(figsize=(12,5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.margins(x=0, y=0)
    plt.show()
    return plt 

In [9]:
#creating a function to show the result of the sentiment analysis from the final df   
def showReport(df):     
    print(f'* the tweets show that the sentiment around "{keyword}" is mainly {df.groupby(by="sentiment").id.count().sort_values(ascending=False).index[0] }')
    print(f'* this is how the overall sentiment and stars ratings breakdown on the {len(df)} total records we recovered : ')
    print(df.groupby(["stars"]).count()['id'])
    
     # Build the percentage of star count reviews by category pie chart.
    star_perc = 100 * df.groupby(["stars"]).count()['id'] / len(df)
    plt.pie(star_perc,
            labels=["1 star", "2 stars", "3 stars", '4 stars', '5 stars'],
            colors=["red", "orange", "gold", 'turquoise', 'green'],
            explode=[0.05, 0.05, 0.05, 0.05, 0.05],
            autopct='%1.1f%%',
            shadow=True, startangle=150),
    plt.title("percentage of Total tweets by star ratings")
    # Show Figure
    plt.show()
     # Build the sentiment reviews by category pie chart.
    sent_perc = 100 * df.groupby(["sentiment"]).count()['id'] / len(df)   
    plt.pie(sent_perc,
            labels=["Negative", "Neutral", "Positive",],
            colors=["red", "gold", 'green'],
            explode=[0.05, 0.05, 0.05],
            autopct='%1.1f%%',
            shadow=True, startangle=150),
    plt.title("percentage of total tweets by sentiment ")
    # Show Figure
    plt.show()
    
#function to creat word clouds for each tweet sentiment
def sentimentWordcloud(df):
    print("We generate Wordclouds for each sentiment to see the words that appear most often for each one :")
    print("_________________________________________________________________________________________________")
    print('Wordcloud for negative sentiment tweets : ')
    creatWordCloud(df.query('sentiment == "negative"'),"clean_tweet")
    print('Wordcloud for neutral sentiment tweets : ')
    creatWordCloud(df.query('sentiment == "neutral"'),"clean_tweet")
    print('Wordcloud for positive sentiment tweets : ')
    creatWordCloud(df.query('sentiment == "positive"'),"clean_tweet")

In [10]:
# creating call functions

def tweetTodf():
    print ("this is a simple twitter sentiment analysis bot, please follow the instuction to know twitter's last thoughts. \n- the tweets collected are the last specified number of tweets about a keyword of your choice")
    print ('connection to twitter IPA')
    df = tweetsETL(getTweets())
    print ('retrieving tweets -- ')
    return df

def sentimentTodf(df):
    print('-----------------------------------------')
    print('sentiment analysis in progress.')
    print('this might take a minute ...')
    final_df = sentimentToDf(df,sentimentAnalysis(df))
    print('-----------------------------------------')
    return final_df

def finalReport(final_df):
    print('creating report ...')
    print(f'the report represents the sentiment around "{keyword}", stars represent the sentiment of a tweet \n from 1 being most negative to 5 being most positive ...')
    print("_________________________________________________________________________________________________\n")
    showReport(final_df)
    sentimentWordcloud(final_df)
    
    

In [ ]:
finalReport(sentimentTodf(tweetTodf()))

this is a simple twitter sentiment analysis bot, please follow the instuction to know twitter's last thoughts. 
- the tweets collected are the last specified number of tweets about a keyword of your choice
connection to twitter IPA
insert search term : high self-esteem
how many tweets do you want? 10000
